In [0]:
#loading the libraries
from collections import deque
from PIL import Image, ImageDraw
import cv2
import numpy as np
import tensorflow as tf
from pycocotools.coco import COCO
from tqdm import tqdm
import sys
import os
import matplotlib.pyplot as plt
from keras.layers import Dense, Input, Convolution2D, Flatten, Reshape, Dropout, MaxPooling2D
from keras.applications import vgg16
from keras.optimizers import SGD
from keras import backend as K
from keras.models import Model, model_from_json
from skimage.io import imread
from keras.engine import Layer

Using TensorFlow backend.


In [0]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip

In [0]:
!unzip /content/val2017.zip
!unzip /content/annotations_trainval2017.zip

In [0]:
 !mv /content/annotations /content/coco

In [0]:
 !mv /content/val2017 /content/images
 !mv /content/images /content/coco  
# !mv /content/coco/instances_val2017.json /content/coco/annotations

In [0]:
# Data Preparation
class data(object):
    def __init__(self, dataDir, max_file_size=150000):
        self.data_path = dataDir
        self.max_file_size = max_file_size


    def get_score(self, ann, img):
        ann_ratio = ann['area'] / (img['width'] * img['height'])

        ann_center = (int(ann['bbox'][0] + ann['bbox'][2] / 2), int(ann['bbox'][1] + ann['bbox'][3] / 2))
        ann_center_bounds = (range(int(img['width'] / 4), int(img['width'] - img['width'] / 4)),
                             range(int(img['height'] / 4), int(img['height'] - img['height'] / 4)))
        ann_centered = ann_center[0] in ann_center_bounds[0] and ann_center[1] in ann_center_bounds[1]

        ann_br = (int(ann['bbox'][0] + ann['bbox'][2]), int(ann['bbox'][1] + ann['bbox'][3]))
        ann_fully_contained = ann['bbox'][0] > 0 and ann['bbox'][1] > 0 and \
                              ann_br[0] < img['width'] and ann_br[1] < img['height']

        return 1 if  ann['iscrowd'] == 0 and ann_ratio > 0.05 and ann_centered and ann_fully_contained else -1


    def create_data(self,data_type):
        coco = COCO('{}/annotations/instances_{}.json'.format(self.data_path, data_type))


        imgIds = coco.getImgIds()

        scores=[]
        masks=[]
        images=[]
        for id in imgIds :
            img = coco.loadImgs(id)[0]
            im_path = '{}/images/{}'.format(self.data_path, img['file_name'])
            image=imread(im_path)
            image = cv2.resize(image, (224, 224)).astype(np.float32)

            annIds = coco.getAnnIds(imgIds=[id], iscrowd=0)
            anns = coco.loadAnns(annIds)
            if (image.shape== (224, 224, 3)):
              for ann in anns:
                  score = self.get_score(ann, img)
                  if score > 0:
                      mask = Image.new('F', (img['width'], img['height']), color=-1)
                      segs = list(zip(*[iter(ann['segmentation'][0])] * 2))
                      ImageDraw.Draw(mask).polygon(segs, outline=1, fill=1)
                      mask = np.asarray(mask)
                      mask = cv2.resize(mask, (224, 224))
                      mask = np.where(mask == -1.0, -1, 1).astype(np.int8)
                      scores.append(score)
                      masks.append(mask)
                      images.append(image)
        return np.asarray(scores),np.asarray(masks),np.asarray(images)

In [0]:
# Loading the data
dataDir = '/content/coco'
dataType = 'val2017'
rc = data(dataDir)
sc,ma,im=rc.create_data(dataType)

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!


In [0]:
# Getting the pretrrained model from VGG16
def initVgg16():
    vgg = vgg16.VGG16(weights="imagenet")
    inp = Input(shape=(224, 224, 3), name='in')
    shared_layers = vgg.layers[1](inp)
    for i in range(len(vgg.layers)):
        if 1 < i < len(vgg.layers) - 5:
            shared_layers = vgg.layers[i](shared_layers)
    return inp, shared_layers

In [0]:
# Upsampling the generated masks
class BilinearUpsampling(Layer):

  def __init__(self, upsampling, data_format=None, **kwargs):

      super(BilinearUpsampling, self).__init__(**kwargs)
      self.upsampling = upsampling

  def call(self, inputs):
    seg=tf.image.resize_bilinear(inputs, (int(inputs.shape[1]*self.upsampling[0]),
                                        int(inputs.shape[2]*self.upsampling[1])))
    seg=tf.squeeze(seg,3)
    return seg
  def compute_output_shape(self, input_shape):
    height = self.upsampling[0] * input_shape[1] if input_shape[1] is not None else None
    width = self.upsampling[1] *input_shape[2] if input_shape[2] is not None else None
    return (input_shape[0], height,width)

In [0]:
# Layers for score predictions and mask generation
inp, shared_layers = initVgg16()
score_predictions = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(shared_layers)
score_predictions = Flatten()(score_predictions)
score_predictions = Dense(512, activation='relu')(score_predictions)
score_predictions = Dropout(0.5)(score_predictions)
# to change in order to the number of classes
score_predictions = Dense(10, activation='relu')(score_predictions)
score_predictions = Dropout(0.5)(score_predictions)
score_predictions = Dense(1, name='score_out')(score_predictions)

seg_predictions = Convolution2D(512, (1, 1), activation='relu')(shared_layers)
seg_predictions = Flatten()(seg_predictions)
seg_predictions = Dense(512)(seg_predictions)
seg_predictions = Dense(56 * 56)(seg_predictions)
seg_predictions = Reshape(target_shape=(56,56,1))(seg_predictions))
seg_predictions = BilinearUpsampling((4,4),name='seg_out')(seg_predictions)

In [0]:
# Loss Functions
def binary_regression_error(y_true, y_pred):
    return 1./32 * K.log(1 + K.exp(-y_true*y_pred))

def mask_binary_regression_error(y_true, y_pred):
    return 0.5 * (1 - y_true[0][0][0]) * K.mean(K.log(1 + K.exp(-y_true*y_pred)))
#  Build the model and compile it
model = Model(inputs=inp, outputs=[seg_predictions, score_predictions])
sgd = SGD(lr=0.001, decay=0.00005, momentum=0.9, nesterov=True, clipvalue=500)
model.compile(optimizer=sgd, loss={'score_out': binary_regression_error, 'seg_out': mask_binary_regression_error},metrics=['accuracy'])

In [0]:
#Training
model.fit({'in': im[:70]}, {'score_out': sc[:70], 'seg_out': ma[:70]}, epochs=500, verbose=1, shuffle=True)

In [0]:
tes=im[:30]

In [0]:
plt.imshow(tes[21]/255)

In [0]:
plt.imshow(tes[21]/255)
plt.imshow(ma[21], alpha = 0.5)
plt.show()

In [0]:
#prediction
re=model.predict(tes)

In [0]:
plt.imshow(tes[21]/255)
plt.imshow(re[0][21], alpha=0.5)
plt.show()

In [0]:
#saving the model in json file
from keras.models import model_from_json
from keras.models import load_model
model_json = model.to_json()

with open("model_num.json", "w") as json_file:
  json_file.write(model_json)

  model.save_weights("model_num.h5")